In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import json
import folium


from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.signal import find_peaks
from scipy.signal import argrelextrema
from plotly.subplots import make_subplots

import geopandas as gpd
from shapely.geometry import Point
from folium.plugins import MarkerCluster

In [2]:
contacts = pd.read_excel('/content/Contacts (Result) (1).xlsx', dtype={'Id': str})
calls = pd.read_excel('/content/Calls (Result) (1).xlsx', dtype={'Id': str, "CONTACTID": str})
spend = pd.read_excel('/content/Spend (Result) (1).xlsx')
deals = pd.read_excel('/content/Deals (Result) (1).xlsx', dtype={'Id': str, 'Contact Name': str})


In [3]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92617 entries, 0 to 92616
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          92617 non-null  object        
 1   Call Start Time             92617 non-null  datetime64[ns]
 2   Call Owner Name             92617 non-null  object        
 3   CONTACTID                   92617 non-null  object        
 4   Call Type                   92617 non-null  object        
 5   Call Duration (in seconds)  92538 non-null  float64       
 6   Call Status                 92617 non-null  object        
 7   Outgoing Call Status        92617 non-null  object        
 8   Scheduled in CRM            92617 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 6.4+ MB


In [4]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19788 entries, 0 to 19787
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   19788 non-null  object        
 1   Deal Owner Name      19788 non-null  object        
 2   Closing Date         13129 non-null  datetime64[ns]
 3   Quality              17557 non-null  object        
 4   Stage                19788 non-null  object        
 5   Lost Reason          14347 non-null  object        
 6   Page                 19788 non-null  object        
 7   Campaign             15555 non-null  object        
 8   Content              13771 non-null  object        
 9   Term                 12077 non-null  object        
 10  Source               19788 non-null  object        
 11  Payment Type         482 non-null    object        
 12  Product              3537 non-null   object        
 13  Education Type       3390 non-n

In [5]:
contacts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Id                  18548 non-null  object        
 1   Contact Owner Name  18548 non-null  object        
 2   Created Time        18548 non-null  datetime64[ns]
 3   Modified Time       18548 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 579.8+ KB


In [6]:
spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14967 entries, 0 to 14966
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         14967 non-null  datetime64[ns]
 1   Source       14967 non-null  object        
 2   Campaign     14967 non-null  object        
 3   Impressions  14967 non-null  int64         
 4   Spend        14967 non-null  float64       
 5   Clicks       14967 non-null  int64         
 6   AdGroup      12894 non-null  object        
 7   Ad           12894 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 935.6+ KB


## Time Series Analysis:
---

### 1. Analyze the trend of deal creation over time and its relationship with calls.



---



To accomplish the assigned tasks, it is necessary to merge the tables. Let's identify the appropriate fields for this.

Optimal fields for merging:  
- **Contact Name (Deals)** and **Id (Contacts)**: to link deal information with contacts.  
- **CONTACTID (Calls)** and **Id (Contacts)**: to link call information with contacts.  

We will use an **outer** join type to merge the tables, ensuring that as much information as possible is retained, even if some data is missing.

In [7]:
deals_contacts = pd.merge(deals, contacts, left_on='Contact Name', right_on='Id',
                          how='left')
calls_contacts = pd.merge(calls, contacts, left_on='CONTACTID', right_on='Id',
                          how='left')
deals_spend = pd.merge(deals, spend, on='Campaign', how='outer')

In [8]:
deals_contacts.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19788 entries, 0 to 19787
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id_x                 19788 non-null  object        
 1   Deal Owner Name      19788 non-null  object        
 2   Closing Date         13129 non-null  datetime64[ns]
 3   Quality              17557 non-null  object        
 4   Stage                19788 non-null  object        
 5   Lost Reason          14347 non-null  object        
 6   Page                 19788 non-null  object        
 7   Campaign             15555 non-null  object        
 8   Content              13771 non-null  object        
 9   Term                 12077 non-null  object        
 10  Source               19788 non-null  object        
 11  Payment Type         482 non-null    object        
 12  Product              3537 non-null   object        
 13  Education Type       3390 non-n

In [9]:
calls_contacts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92617 entries, 0 to 92616
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id_x                        92617 non-null  object        
 1   Call Start Time             92617 non-null  datetime64[ns]
 2   Call Owner Name             92617 non-null  object        
 3   CONTACTID                   92617 non-null  object        
 4   Call Type                   92617 non-null  object        
 5   Call Duration (in seconds)  92538 non-null  float64       
 6   Call Status                 92617 non-null  object        
 7   Outgoing Call Status        92617 non-null  object        
 8   Scheduled in CRM            92617 non-null  object        
 9   Id_y                        88815 non-null  object        
 10  Contact Owner Name          88815 non-null  object        
 11  Created Time                88815 non-null  datetime64


### Aggregating Data by Month for Calls and Deals


In [10]:
deals_by_time = deals_contacts.resample(
            'ME', on='Created Time_y').size().reset_index(name='Deals Count')
calls_by_time = calls_contacts.resample(
            'ME', on='Call Start Time').size().reset_index(name='Calls Count')

In [11]:
deals_by_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Created Time_y  12 non-null     datetime64[ns]
 1   Deals Count     12 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 324.0 bytes


In [12]:
calls_by_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Call Start Time  13 non-null     datetime64[ns]
 1   Calls Count      13 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 340.0 bytes


### We combine the tables obtained after aggregation

In [13]:
time_series_data = pd.merge(deals_by_time, calls_by_time,
                            left_on='Created Time_y',
                            right_on='Call Start Time',
                            how='outer')
time_series_data = time_series_data.rename(columns={'Created Time_y': 'Date'})

In [14]:
time_series_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             12 non-null     datetime64[ns]
 1   Deals Count      12 non-null     float64       
 2   Call Start Time  13 non-null     datetime64[ns]
 3   Calls Count      13 non-null     int64         
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 548.0 bytes


### Create a visualization of the trend of creating deals and calls by time

-- by month

In [15]:
fig = px.line(time_series_data,
              x='Date',
              y=['Deals Count', 'Calls Count'],
              title='Monthly trend of creating deals and calls by time',
              labels={'value': 'Quantity', 'variable': ''},
              template='plotly_white', markers=True)

for i, column in enumerate(['Deals Count', 'Calls Count']):
    fig.data[i].update(
        text=time_series_data[column],
        textposition="top center",
        mode="lines+markers+text"
    )

fig.update_layout(
    title_x=0.5,
    xaxis_title="",
    legend=dict(
    orientation="h",
    x=0.5,
    xanchor="center",
    y=1.1
    )
)

fig.show()

### 🔍 Conclusion:
- The graph shows that the number of calls and the number of deals have different dynamics over the analyzed period.
- The number of calls demonstrates significant growth, peaking in May 2024, followed by a decline.
- Meanwhile, the number of deals also increases but more gradually, with less pronounced peaks.
- The maximum number of deals also occurs in May 2024, which may indicate a correlation between call activity and the number of deals closed.
- The total number of deals is noticeably lower compared to the number of calls, highlighting the importance of effectively converting calls into successful deals.

📊 For a more detailed analysis, we will plot the time series at weekly intervals and then decompose it to identify trends and seasonality. Plotting the time series at two-week intervals allows us to better see changes and trends in the data, as the smaller interval gives a more detailed view of the data.


In [16]:
deals_by_time_w = deals_contacts.resample(
            'W', on='Created Time_y').size().reset_index(name='Deals Count')
calls_by_time_w = calls_contacts.resample(
            'W', on='Call Start Time').size().reset_index(name='Calls Count')


In [17]:
time_series_data_w = pd.merge(deals_by_time_w, calls_by_time_w,
                            left_on='Created Time_y',
                            right_on='Call Start Time',
                            how='outer')
time_series_data_w = time_series_data_w.rename(columns={'Created Time_y': 'Date'})

In [18]:
def find_extrema(series):
    indexes_max = argrelextrema(series.values, np.greater, order=2)[0]
    return indexes_max

fig = px.line(time_series_data_w,
              x='Date',
              y=['Deals Count', 'Calls Count'],
              title='Weekly trend of creating deals and calls by time',
              labels={'value': 'Quantuty', 'variable': ''},
              template='plotly_white', markers=True)

for i, column in enumerate(['Deals Count', 'Calls Count']):
    indexes_max = find_extrema(time_series_data_w[column].fillna(0))
    text_labels = ['' if idx not in indexes_max else f'{int(val)}'
                   for idx, val in enumerate(time_series_data_w[column])]

    fig.data[i].update(
        text=text_labels,
        textposition="top center",
        mode="lines+markers+text"
    )

    x_vals = np.arange(len(time_series_data_w))
    y_vals = time_series_data_w[column].fillna(0).values
    trend = np.polyfit(x_vals, y_vals, deg=1)
    trend_line = np.polyval(trend, x_vals)

    fig.add_scatter(x=time_series_data_w['Date'], y=trend_line, mode='lines',
                    name=f'{column} Trend', line=dict(dash='dot'))

fig.update_layout(
    title_x=0.5,
    xaxis_title="",
    legend=dict(
    orientation="h",
    x=0.5,
    xanchor="center",
    y=1.1
    )
)

fig.show()

🔍 **Overall Conclusion for Both Graphs:**

- The global trend (monthly analysis) shows steady growth in calls and deals until May 2024, followed by a sharp decline.
- A detailed analysis (weekly) reveals cyclicality — growth every 3-4 weeks.
- Peaks in calls usually precede peaks in deals, which is logical (calls lead to deal conversions).
- The decline after May 2024 requires further analysis — it may be related to seasonality, changes in marketing activities, or other factors.


📝 **Recommendations:**

- Analyze what happened in May 2024 — whether there was a marketing campaign, discounts, or changes in business strategy.
- Study the 3-4 week cycle and try to optimize it (increase the number of calls during downturns).
- Investigate the reasons for the sharp decline after May 2024 and explore potential solutions.



---



In the previous graphs, general trends in deals and calls were identified, including their growth and seasonal fluctuations. However, for a deeper analysis of the factors influencing the dynamics of deals and calls, it is important to consider additional variables, such as:

- **Contacts Count**: This metric reflects the number of new potential clients. An increase in contacts may indicate growing interest in the product.
- **Ad Spend**: Analyzing the budget allocated for customer acquisition and its impact on deals and calls.

In [19]:
def transfer_to_week(df, time_column, target_column=None):
    df_copy = df.copy()
    result = pd.DataFrame()

    if target_column is not None:
        weekly_time_series = (
            df_copy.set_index(time_column)
            .resample('W')[target_column]
            .sum()
            .reset_index()
        )
        result['Date'] = weekly_time_series[time_column]
        result['Target'] = weekly_time_series[target_column]

    else:
        weekly_time_series = (
            df_copy.set_index(time_column)
            .resample('W')
            .size()
            .reset_index()
        )
        result['Date'] = weekly_time_series[time_column]
        result['Target'] = weekly_time_series[0]

    return result

In [20]:
spend_w = transfer_to_week(spend,'Date', 'Spend')
contacts_w = transfer_to_week(contacts, 'Created Time')
deals_w = transfer_to_week(deals, 'Created Time')
calls_w = transfer_to_week(calls, 'Call Start Time')

In [21]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=deals_w['Date'],
                         y=deals_w['Target'],
                         mode='lines+markers',
                         name='Deals Count (Weekly)'))

fig.add_trace(go.Scatter(x=calls_w['Date'],
                         y=calls_w['Target'],
                         mode='lines+markers',
                         name='Calls Count (Weekly)'))

fig.add_trace(go.Scatter(x=contacts_w['Date'],
                         y=contacts_w['Target'],
                         mode='lines+markers',
                         name='Contacts Count (Weekly)',
                         line=dict(dash='dash')))

fig.add_trace(go.Scatter(x=spend_w['Date'],
                         y=spend_w['Target'],
                         mode='lines+markers',
                         name='Ad Spend (Weekly)',
                         line=dict(dash='dot')))

fig.update_layout(
    title='Weekly Trends of Deals, Calls, Contacts, and Ad Spend',
    title_x=0.5,
    yaxis_title='Count / Spend',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        xanchor="center",
        y=1.1
    ),
    xaxis=dict(tickangle=0),
    template='plotly_white'
)

fig.show()

## 🔍 Insights from the Graph

- **The growth in deals and calls largely coincides with the increase in marketing expenses.**  
  There are periods where a rise in advertising costs precedes an increase in the number of calls and deals. This confirms that marketing activity plays a key role in attracting customers.

- **Contacts show a more stable but modest growth.**  
  The number of new contacts is increasing, but not as sharply as calls and deals. This may indicate that the most interested clients have already been reached, and the efficiency of acquiring new contacts is starting to decline.

- **Cyclicality in marketing expenses and their impact on other metrics is observed.**  
  Every few weeks, peaks in advertising expenses are followed by peaks in calls and deals. This suggests that the marketing strategy is working but requires an analysis of spending efficiency.

- **The decline after May 2024 is also evident in this graph.**  
  After the peak in May, there is a sharp drop in all metrics, including calls, deals, and advertising expenses. It is important to determine the cause — whether it is due to a change in strategy, seasonal factors, or budget reallocation.

In [22]:
time_series_data_cleaned_w = time_series_data_w.dropna(subset=['Deals Count'])
decomposition_deals = seasonal_decompose(
    time_series_data_cleaned_w['Deals Count'],
    model='multiplicative', period=12
    )

trend_deals = decomposition_deals.trend
seasonal_deals = decomposition_deals.seasonal
residual_deals = decomposition_deals.resid


decomposition_calls = seasonal_decompose(
    time_series_data_cleaned_w['Calls Count'],
    model='multiplicative', period=12
    )

trend_calls = decomposition_calls.trend
seasonal_calls = decomposition_calls.seasonal
residual_calls = decomposition_calls.resid

In [23]:
def plot_series_subplot(fig, series, title, color, row, col):
    if series is not None:
        fig.add_trace(
            go.Scatter(
                x=time_series_data_cleaned_w.index,
                y=series, mode="lines", name=title, line=dict(color=color)),
            row=row,
            col=col,
        )

fig = make_subplots(rows=3, cols=2, subplot_titles=(
    "Trend - Deals Count", "Trend - Calls Count",
    "Seasonality - Deals Count", "Seasonality - Calls Count",
    "Residuals - Deals Count", "Residuals - Calls Count"
))

plot_series_subplot(fig, trend_deals, "Trend - Deals Count", "blue", 1, 1)
plot_series_subplot(fig, trend_calls, "Trend - Calls Count", "green", 1, 2)

plot_series_subplot(fig, seasonal_deals, "Seasonality - Deals Count", "blue", 2, 1)
plot_series_subplot(fig, seasonal_calls, "Seasonality - Calls Count", "green", 2, 2)

plot_series_subplot(fig, residual_deals, "Residuals - Deals Count", "blue", 3, 1)
plot_series_subplot(fig, residual_calls, "Residuals - Calls Count", "green", 3, 2)

fig.update_layout(height=900, width=1200, showlegend=False)
fig.show()

fig_initial = make_subplots(rows=1, cols=2, subplot_titles=(
    "Initial Data - Deals Count", "Initial Data - Calls Count"
))

plot_series_subplot(
    fig_initial, time_series_data_cleaned_w["Deals Count"],
    "Initial Data - Deals Count", "blue", 1, 1)
if "Calls Count" in time_series_data_cleaned_w.columns:
    plot_series_subplot(
        fig_initial, time_series_data_cleaned_w["Calls Count"],
        "Initial Data - Calls Count", "green", 1, 2)

fig_initial.update_layout(height=400, width=1200, showlegend=False)
fig_initial.show()

### 🔍 Conclusions:

**General Trend:**
 - For calls, there is a positive trend, indicating an increase in the number of calls over time.
   This may suggest growing customer activity or intensified marketing efforts.
 - For deals, there is also a positive trend, but it is less pronounced, indicating more stable growth compared to calls.

**Seasonality:**
 - Both time series show pronounced seasonality with periodic fluctuations every two weeks.
   This could be related to regular marketing campaigns, promotions, or other factors influencing customer behavior.
 - Since seasonal fluctuations are present, it is recommended to plan marketing campaigns in alignment with these cycles to maximize target audience reach.

**Residuals:**
 - Residuals show random fluctuations that cannot be explained by the trend or seasonality.
 - The residual time series for calls and deals indicate the presence of irregular events that may impact the number of calls and deals.
 - Investigating factors that contribute to these random fluctuations can help reduce their impact and improve the stability of the metrics.

---
### 2. Analyzing Deal Closing Time and Duration from Creation to Closing
---

**Analyzing Outliers in Deal Closing Dates (Closing Date Outliers):**  
   - We create a plot to identify anomalous values in deal closing dates. This helps detect unrealistic or erroneous data (e.g., deals with dates outside reasonable timeframes).  
   - Removing or adjusting such outliers improves data quality and makes the analysis more accurate.

In [24]:
min_date = deals_contacts['Closing Date'].min()
max_date = deals_contacts['Closing Date'].max()
print(min_date, max_date)

2022-10-11 00:00:00 2024-12-11 00:00:00


In [25]:
min_date1 = deals['Created Time'].min()
max_date1 = deals['Created Time'].max()
print(min_date1, max_date1)

2023-07-03 17:03:00 2024-06-21 15:30:00


In [26]:
fig = px.box(deals_contacts, x='Closing Date', title='Closing Date')

fig.update_layout(
    xaxis_title="Closing Date Outliers",
    yaxis_title="",
    showlegend=False,
    template="plotly_white"
)

fig.show()


In [27]:
deals_contacts = deals_contacts[deals_contacts['Closing Date'] >= min_date1]

In [28]:
fig = px.box(deals_contacts, x='Closing Date', title='Closing Date')

fig.update_layout(
    xaxis_title="Closing Date Outliers",
    yaxis_title="",
    showlegend=False,
    template="plotly_white"
)

fig.show()


**Analyzing the Distribution of Deal Durations (Duration):**  
- We calculate the duration of deals
- We create a plot to identify anomalous values
- Using the IQR (Interquartile Range) method allows us to objectively determine and remove outliers, focusing on typical values.

In [29]:
deals_contacts['Duration'] = abs((deals_contacts['Closing Date'] - deals_contacts['Created Time_x']).dt.days)

In [30]:
deals_contacts.columns

Index(['Id_x', 'Deal Owner Name', 'Closing Date', 'Quality', 'Stage',
       'Lost Reason', 'Page', 'Campaign', 'Content', 'Term', 'Source',
       'Payment Type', 'Product', 'Education Type', 'Created Time_x',
       'Course duration', 'Months of study', 'Initial Amount Paid',
       'Offer Total Amount', 'Contact Name', 'City', 'Level of Deutsch',
       'SLA Seconds', 'Id_y', 'Contact Owner Name', 'Created Time_y',
       'Modified Time', 'Duration'],
      dtype='object')

In [31]:
deals_contacts['Duration'].value_counts()

,count
Duration,
1,3467
0,1594
2,938
3,778
4,622
...,...
230,1
224,1
176,1


In [32]:
fig = px.histogram(deals_contacts, x='Duration', title='Duration', nbins=30)

fig.update_layout(
    xaxis_title="Duration",
    yaxis_title="Count",
    showlegend=False,
    template="plotly_white"
)

fig.show()

In [33]:
Q1 = deals_contacts['Duration'].quantile(0.25)
Q3 = deals_contacts['Duration'].quantile(0.75)
IQR = Q3 - Q1

lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR

outliers = deals_contacts[(deals_contacts['Duration'] < lower_whisker) |
                          (deals_contacts['Duration'] > upper_whisker)]
num_outliers = len(outliers)

total_points = len(deals_contacts)

outlier_percentage = (num_outliers / total_points) * 100

print(f"Number of outliers: {num_outliers} out of {total_points} records")
print(f"Percentage of outliers: {outlier_percentage:.2f}%")

Number of outliers: 1930 out of 13127 records
Percentage of outliers: 14.70%


📊 **Visualizing the Distribution of Deal Closing Dates (Distribution of Deal Closing Dates):**  

In [34]:
deals_contacts_filtered = deals_contacts[
    (deals_contacts['Duration'] >= lower_whisker) &
    (deals_contacts['Duration'] <= upper_whisker)]

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=[
        "Distribution of Deal Closing Dates",
        "Distribution of Deal Durations",
    ],
)

fig.add_trace(
    go.Histogram(
        x=deals_contacts_filtered["Closing Date"],
        nbinsx=30,
        marker_color="skyblue",
        name="Closing Date",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Histogram(
        x=deals_contacts_filtered["Duration"],
        nbinsx=30,
        marker_color="salmon",
        name="Duration",
    ),
    row=1,
    col=2,
)

closing_date_min = deals_contacts_filtered["Closing Date"].min()
closing_date_max = deals_contacts_filtered["Closing Date"].max()
duration_min = deals_contacts_filtered["Duration"].min()
duration_max = deals_contacts_filtered["Duration"].max()

fig.update_xaxes(
    range=[closing_date_min - pd.Timedelta(days=10),
           closing_date_max + pd.Timedelta(days=10)],
    row=1,
    col=1,
)
fig.update_xaxes(range=[duration_min - 1, duration_max + 1], row=1, col=2)

fig.update_layout(
    height=500,
    bargap=0.1,
    showlegend=False,
)

fig.update_xaxes(title_text="Closing Date", row=1, col=1)
fig.update_yaxes(title_text="Number of Deals", row=1, col=1)

fig.update_xaxes(title_text="Duration (days)", row=1, col=2)
fig.update_yaxes(title_text="Number of Deals", row=1, col=2)

fig.show()

🔍 **Conclusion:**
- **The distribution plot of deal closing dates** shows that the number of closed deals has been increasing since the beginning of 2024, peaking in February and April 2024. In other months, the number of closed deals is more evenly distributed but with fewer deals per month. This may indicate seasonal fluctuations or the influence of specific factors during these periods that contribute to the increase in closed deals.

- **The distribution of the duration from deal creation to closing** reveals that most deals are closed within the first 10 days after their creation, indicating high sales process efficiency. However, there are a few deals that take much longer to close, up to 140 days, although such cases are rare. The majority of deals are closed relatively quickly, suggesting effective customer handling processes, but the longer closing periods for some deals require further analysis to improve these processes.



---



# Campaign Effectiveness Analysis:
---
1. Compare the effectiveness of different campaigns in terms of lead generation and conversion rates.
 - Determining the total number of leads for each source.
 - Calculating the number of successful deals (status: "Payment Done").
 - Calculating the conversion rate (the ratio of successful deals to the number of leads in percent).
- Visualization:
 - Column chart of the number of leads by source.
 - Line chart of the conversion rate.

2. Evaluate the effectiveness of various marketing sources (Source) in generating high-quality leads.

In [35]:
spend[['Campaign', 'Source']] = spend[['Campaign', 'Source']].astype('category')

In [36]:
deals_spend = pd.merge(deals, spend, on='Campaign', how='outer')

In [37]:
deals_spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9533117 entries, 0 to 9533116
Data columns (total 30 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Id                   object        
 1   Deal Owner Name      object        
 2   Closing Date         datetime64[ns]
 3   Quality              object        
 4   Stage                object        
 5   Lost Reason          object        
 6   Page                 object        
 7   Campaign             object        
 8   Content              object        
 9   Term                 object        
 10  Source_x             object        
 11  Payment Type         object        
 12  Product              object        
 13  Education Type       object        
 14  Created Time         datetime64[ns]
 15  Course duration      float64       
 16  Months of study      float64       
 17  Initial Amount Paid  float64       
 18  Offer Total Amount   float64       
 19  Contact Name         

In [38]:
deals_spend.describe()

,Closing Date,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,SLA Seconds,Date,Impressions,Spend,Clicks
count,7128306,9531398,1.777984e+06,382798.000000,2.048586e+06,2.056443e+06,8.228942e+06,9523056,9.523056e+06,9.523056e+06,9.523056e+06
mean,2024-01-19 06:34:21.602405888,2024-01-15 04:15:19.250308096,1.038202e+01,5.707642,8.746182e+02,7.242521e+03,9.828704e+04,2024-01-06 07:24:55.238333952,2.826206e+03,7.029430e+00,2.153452e+01
min,2022-10-11 00:00:00,2023-07-03 17:03:00,6.000000e+00,0.000000,0.000000e+00,0.000000e+00,3.000000e+00,2023-07-03 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2023-11-02 00:00:00,2023-11-01 07:20:00,1.100000e+01,3.000000,3.000000e+02,3.500000e+03,4.395000e+03,2023-10-06 00:00:00,8.100000e+01,6.600000e-01,1.000000e+00
50%,2024-01-24 00:00:00,2024-01-16 17:14:00,1.100000e+01,6.000000,1.000000e+03,1.100000e+04,2.014700e+04,2024-01-16 00:00:00,4.870000e+02,2.880000e+00,5.000000e+00
75%,2024-04-10 00:00:00,2024-04-02 21:09:00,1.100000e+01,8.000000,1.000000e+03,1.100000e+04,5.374500e+04,2024-04-02 00:00:00,1.780000e+03,9.120000e+00,1.800000e+01
max,2024-12-11 00:00:00,2024-06-21 15:30:00,1.100000e+01,11.000000,1.100000e+04,1.150000e+04,2.690846e+07,2024-06-21 00:00:00,4.314450e+05,7.740000e+02,2.415000e+03
std,NaN,NaN,1.645595e+00,2.976320,1.146603e+03,4.648811e+03,7.374416e+05,NaN,9.201501e+03,1.386992e+01,6.523347e+01


In [39]:
deals_spend.head()

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,Content,Term,...,City,Level of Deutsch,SLA Seconds,Date,Source_y,Impressions,Spend,Clicks,AdGroup,Ad
0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,01.02.24wide_webinar_DE,NaN,NaN,...,NaN,NaN,NaN,2024-02-03,Webinar,2927.0,33.14,24.0,wide,v1webinar
1,NaN,NaN,NaT,NaN,NaN,NaN,NaN,01.02.24wide_webinar_DE,NaN,NaN,...,NaN,NaN,NaN,2024-02-03,Webinar,771.0,6.40,4.0,wide,v2webinar
2,NaN,NaN,NaT,NaN,NaN,NaN,NaN,01.02.24wide_webinar_DE,NaN,NaN,...,NaN,NaN,NaN,2024-02-04,Webinar,3076.0,33.78,33.0,wide,v1webinar
3,NaN,NaN,NaT,NaN,NaN,NaN,NaN,01.02.24wide_webinar_DE,NaN,NaN,...,NaN,NaN,NaN,2024-02-04,Webinar,182.0,1.52,2.0,wide,v2webinar
4,NaN,NaN,NaT,NaN,NaN,NaN,NaN,01.02.24wide_webinar_DE,NaN,NaN,...,NaN,NaN,NaN,2024-02-05,Webinar,801.0,7.72,10.0,wide,v2webinar


In [40]:
spend[['Campaign', 'Source']] = spend[['Campaign', 'Source']].astype('category')

In [41]:
source_leads = deals_spend[deals_spend['Source_y'] != 'Test'].groupby('Source_y', observed=True).size().reset_index(name='Leads')
source_leads

,Source_y,Leads
0,Bloggers,258
1,CRM,78
2,Facebook Ads,4577580
3,Google Ads,152861
4,Offline,7
5,Organic,230
6,Partnership,14
7,Radio,1
8,SMM,175
9,Telegram posts,468


In [42]:
spend[spend['Source'] == 'Test']

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
5748,2023-12-08,Test,07.12.23test_DE,93,1.37,0,wide_webdesigner,b3webdesigner
5752,2023-12-08,Test,07.12.23test_DE,2,0.00,0,wide_python-developer,b1python-developer
5759,2023-12-08,Test,07.12.23test_DE,17,0.22,0,wide_qa-engineer,b1qa-engineer
5761,2023-12-08,Test,07.12.23test_DE,13,0.26,0,wide_python-developer,b3python-developer
5764,2023-12-08,Test,07.12.23test_DE,153,2.21,4,wide_python-developer,b4python-developer
...,...,...,...,...,...,...,...,...
13011,2024-05-11,Test,02.05.24test_DE,1,0.02,0,accountant_wide,b2accountant
13019,2024-05-11,Test,02.05.24test_DE,1888,15.59,76,accountant_wide,b3accountant
13038,2024-05-12,Test,02.05.24test_DE,2698,18.36,83,accountant_wide,b3accountant
13041,2024-05-12,Test,02.05.24test_DE,1,0.02,0,accountant_wide,b1accountant


In [43]:
spend[spend['Source'] == 'Test'].describe()

,Date,Impressions,Spend,Clicks
count,169,169.000000,169.000000,169.000000
mean,2024-01-01 09:30:53.254437888,260.171598,3.598876,7.254438
min,2023-12-08 00:00:00,1.000000,0.000000,0.000000
25%,2023-12-11 00:00:00,14.000000,0.310000,0.000000
50%,2023-12-16 00:00:00,58.000000,1.300000,1.000000
75%,2023-12-19 00:00:00,275.000000,5.840000,6.000000
max,2024-05-13 00:00:00,2698.000000,18.920000,110.000000
std,NaN,494.554104,4.650666,18.949818


In [44]:
spend['Source'].value_counts()

,count
Source,
Facebook Ads,8730
Tiktok Ads,2200
Youtube Ads,1423
Google Ads,911
Telegram posts,468
Webinar,303
Bloggers,258
Organic,230
SMM,175


In [45]:
successful_deals = deals_spend[(deals_spend['Source_y'] != 'Test')
                             & (deals_spend['Stage'] == 'Payment Done')\
                               ].groupby('Source_y', observed=True).size().reset_index(name='Successful Deals')

successful_deals

,Source_y,Successful Deals
0,Facebook Ads,230271
1,Google Ads,3682
2,Tiktok Ads,87765
3,Webinar,533
4,Youtube Ads,64607


In [46]:
source_performance = pd.merge(source_leads, successful_deals, on='Source_y',
                              how='outer')

source_performance['Successful Deals'] = source_performance[
                        'Successful Deals'].fillna(0)
source_performance['Conversion Rate'] = source_performance[
                        'Successful Deals'] /source_performance['Leads'] * 100
source_performance

,Source_y,Leads,Successful Deals,Conversion Rate
0,Bloggers,258,0.0,0.000000
1,CRM,78,0.0,0.000000
2,Facebook Ads,4577580,230271.0,5.030409
3,Google Ads,152861,3682.0,2.408724
4,Offline,7,0.0,0.000000
5,Organic,230,0.0,0.000000
6,Partnership,14,0.0,0.000000
7,Radio,1,0.0,0.000000
8,SMM,175,0.0,0.000000
9,Telegram posts,468,0.0,0.000000


In [47]:
fig = go.Figure()

source_performance = source_performance.sort_values(
    by='Leads', ascending=False)

fig.add_trace(go.Bar(
    x=source_performance['Source_y'],
    y=source_performance['Leads'],
    name='Leads',
    marker_color='skyblue',
    text=source_performance['Leads'],
    textposition='outside'
))

fig.add_trace(go.Scatter(
    x=source_performance['Source_y'],
    y=source_performance['Conversion Rate'],
    name='Conversion Rate (%)',
    mode='lines+markers+text',
    text=[f"{y:.1f}%" for y in source_performance['Conversion Rate']],
    textposition='top center',
    marker=dict(color='red', size=8),
    line=dict(color='red', width=2, dash='solid'),
    yaxis='y2'
))

fig.update_layout(
    title=dict(
        text="Source Performance: Leads and Conversion Rate",
        x=0.5,
        xanchor="center",
        font=dict(size=18)
    ),
    xaxis=dict(title="Source", tickangle=-45),
    yaxis=dict(title="Number of Leads", showgrid=False),
    yaxis2=dict(
        title="Conversion Rate (%)",
        overlaying='y',
        side='right',
        showgrid=False,
        range=[-1, 20]
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5
    ),
    height=600,
    width=1000,
    template="plotly_white"
)

fig.show()

### 🔍 Insights from the Graph:
1. The highest number of leads comes from Facebook Ads (4.57M), TikTok Ads (2.79M), and YouTube Ads (1.95M).  
2. The highest conversion rates are observed for Facebook Ads (5.0%), YouTube Ads (3.3%), and TikTok Ads (3.1%).  
3. Google Ads (152K leads) has a low conversion rate (2.4%), which is lower than the top 3 sources.  
4. Other sources generate few leads, and Telegram, Bloggers, Organic, SMM, CRM, Partnership, Offline, and Radio show 0% conversion.  

**Conclusion:** It is worth focusing on Facebook, TikTok, and YouTube Ads as the most effective acquisition channels.

## Evaluate the effectiveness of different marketing sources in generating quality leads.
---

### 📊 For a more detailed analysis, we will build a heatmap for marketing channels considering 4 metrics to analyze the correlation between these metrics:

******

*   Number of clicks
*   Advertising costs

*   Total sales amount
*   Number of successful deals


In [48]:
filtered_sources = [
    "Facebook Ads", "Tiktok Ads", "Youtube Ads", "Google Ads", "Webinar"
                   ]
filtered_deals = deals_spend[deals_spend['Source_y']\
                             .astype(str).isin(filtered_sources)]

In [49]:
filtered_deals['Source_y'].value_counts()

,count
Source_y,
Facebook Ads,4577580
Tiktok Ads,2799887
Youtube Ads,1947599
Google Ads,152861
Webinar,41230
Bloggers,0
CRM,0
Offline,0
Organic,0


In [50]:
deals_spend['Source_y'].unique()

['Webinar', 'Facebook Ads', 'Test', 'Tiktok Ads', NaN, ..., 'CRM', 'Offline', 'Partnership', 'Radio', 'Youtube Ads']
Length: 15
Categories (14, object): ['Bloggers', 'CRM', 'Facebook Ads', 'Google Ads', ..., 'Test', 'Tiktok Ads',
                          'Webinar', 'Youtube Ads']

In [51]:
filtered_df = deals_spend[deals_spend['Source_y'].isin(filtered_sources)]

heatmap_data = filtered_df.groupby('Source_y', observed=False).agg({
    'Clicks': 'sum',
    'Spend': 'sum',
    'Offer Total Amount': 'sum',
    'Stage': lambda x: (x == 'Payment Done').sum()
}).rename(columns={'Stage': 'Successful Deals'}).reset_index()

heatmap_data = heatmap_data[heatmap_data['Source_y'].isin(filtered_sources)]

cols = ['Clicks', 'Spend', 'Offer Total Amount', 'Successful Deals']
heatmap_data[cols] = heatmap_data[cols].astype(float)

heatmap_data.set_index('Source_y', inplace=True)
heatmap_data_normalized = heatmap_data / heatmap_data.max()

fig = px.imshow(
    heatmap_data_normalized,
    labels=dict(x="Metrics", y="Source", color="Normalized Value"),
    x=heatmap_data_normalized.columns,
    y=heatmap_data_normalized.index,
    color_continuous_scale="PuBu",
    text_auto=".2f"
)

fig.update_layout(
    title=dict(
        text="Heatmap: Clicks, Spend, Revenue, and Successful Deals by Source",
        x=0.465,
        xanchor="center"
    ),
    xaxis_title="Metrics",
    yaxis_title="Source",
    width=1200,
    height=800,
    xaxis=dict(tickangle=0)
)

fig.show()

### 🔍 Key observations:
Based on the conducted analysis, the following conclusions can be drawn.

1. **Facebook Ads**  
   - High correlation between Spend (0.80) and Clicks (0.30).  
   - Perfect correlation (1.00) between Offer Total Amount and Successful Deals.  
   - This indicates that all spent money is directly converted into revenue and successful deals.  

2. **Google Ads**  
   - Moderate correlation between Clicks (0.53) and Spend (0.49).  
   - Almost no correlation with Offer Total Amount (0.02) and Successful Deals (0.02).  
   - This may indicate that Google Ads generates clicks, but they do not lead to significant revenue or deals.  

3. **TikTok Ads**  
   - Moderate correlation between Clicks (0.38) and Spend (0.65).  
   - Moderate correlation with Offer Total Amount (0.44) and Successful Deals (0.38).  
   - This suggests that TikTok delivers moderate results in terms of both customer acquisition and revenue.  

4. **Webinar**  
   - Almost zero correlation with all metrics.  
   - This indicates that webinars do not generate significant traffic, expenses, revenue, or successful deals.  

5. **YouTube Ads**  
   - Clicks (1.00) and Spend (1.00) → perfect dependency (logical, as no budget means no clicks).  
   - Low correlation with Offer Total Amount (0.29) and Successful Deals (0.28).  
   - This means that YouTube generates traffic but does not always lead to successful deals.  

### 📝 Recommendations:
- **Facebook Ads** is the most effective source, as investments (Spend) are directly linked to revenue and successful deals. Consider increasing the budget.  
- **Google Ads and YouTube Ads** generate clicks but do not lead to deals. A traffic quality analysis is required. It may be necessary to adjust targeting or creatives.  
- **TikTok Ads** delivers average results. A/B testing could be conducted to identify ways to increase revenue.  
- **Webinar** is completely ineffective. Consider either discontinuing or revising the webinar strategy.

---


# **Sales Team Performance Analysis:**
1. Evaluate the effectiveness of individual deal owners and advertising campaigns in terms of the number of processed deals, conversion rates, and total sales amounts.

- Analyze the number of processed deals to understand how many deals each owner has handled, assessing their workload and productivity.

- Analyze the number of successful deals (status "Payment Done") to evaluate how effective owners are at closing deals.

- Analyze the total sales amount, which represents the total value of offers that were successfully closed. This allows us to measure each owner's financial contribution to overall sales.

- Analyze the conversion rate, which shows the percentage of successful deals relative to the total number of processed deals, providing insight into the deal owner's effectiveness.



---



### Calculation of Deal Owners' Performance Metrics
 -- Calculating the number of processed deals, successful deals, and total payment amounts --

In [52]:
owner_deals = deals.groupby(
    'Deal Owner Name',
    observed=True)['Id'].nunique().reset_index(name='Total Deals')

successful_owner_deals = (
    deals[deals['Stage'] == 'Payment Done']
    .groupby('Deal Owner Name', observed=True)['Id']
    .nunique()
    .reset_index(name='Successful Deals')
)

total_sales = deals[
    deals['Stage'] == 'Payment Done'].groupby(
        'Deal Owner Name', observed=True)['Offer Total Amount'].sum().reset_index()


In [53]:
owner_performance = pd.merge(
    owner_deals, successful_owner_deals, on='Deal Owner Name', how='left')
owner_performance = pd.merge(
    owner_performance, total_sales, on='Deal Owner Name', how='left')

owner_performance.fillna(
    {'Successful Deals': 0, 'Offer Total Amount': 0}, inplace=True)

owner_performance['Conversion Rate'] = owner_performance[
                  'Successful Deals'] / owner_performance['Total Deals'] * 100

owner_performance_sorted = owner_performance.sort_values(
                    by='Successful Deals', ascending=False)

display(owner_performance_sorted.style.format({
    'Successful Deals': '{:,.0f}',
    'Offer Total Amount': '{:,.0f}',
    'Conversion Rate': '{:.2f}'
}).hide(axis='index'))

Deal Owner Name,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
Charlie Davis,2797,148,"1,066,600",5.29
Ulysses Adams,2069,141,"1,011,400",6.81
Paula Underwood,1771,93,"694,000",5.25
Julia Nelson,2085,92,"718,701",4.41
Quincy Vincent,1805,65,"465,500",3.60
Oliver Taylor,160,50,"524,500",31.25
Ben Hall,1303,46,"345,500",3.53
Nina Scott,1217,46,"315,000",3.78
Victor Barnes,1187,44,"356,000",3.71
Kevin Parker,500,40,"211,900",8.00


In [54]:
owner_performance_sorted_conversion = owner_performance.sort_values(
                    by='Conversion Rate', ascending=False)

owner_performance_sorted_conversion.head()

,Deal Owner Name,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
16,Oliver Taylor,160,50.0,524500.0,31.250000
11,John Doe,12,1.0,0.0,8.333333
13,Kevin Parker,500,40.0,211900.0,8.000000
21,Ulysses Adams,2069,141.0,1011400.0,6.814886
5,Charlie Davis,2797,148.0,1066600.0,5.291384


In [55]:
Q1_T = owner_performance['Total Deals'].quantile(0.25)

owner_performance_f_deals = owner_performance[
    owner_performance['Total Deals'] > Q1_T
]

Q1_S = owner_performance_f_deals[
    owner_performance_f_deals['Successful Deals'] > 0
]['Successful Deals'].quantile(0.25)

owner_performance_f_deals = owner_performance_f_deals[
    owner_performance_f_deals['Successful Deals'] > Q1_S
]

owner_performance_f_deals_conversion = owner_performance_f_deals.sort_values(
    by='Conversion Rate', ascending=False
)

display(owner_performance_f_deals_conversion.style.format({
    'Total Deals': '{:,.0f}',
    'Successful Deals': '{:,.0f}',
    'Offer Total Amount': '{:,.0f}',
    'Conversion Rate': '{:.2f}'
}).hide(axis='index'))


Deal Owner Name,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
Oliver Taylor,160,50,"524,500",31.25
Kevin Parker,500,40,"211,900",8.00
Ulysses Adams,"2,069",141,"1,011,400",6.81
Charlie Davis,"2,797",148,"1,066,600",5.29
Paula Underwood,"1,771",93,"694,000",5.25
Julia Nelson,"2,085",92,"718,701",4.41
Nina Scott,"1,217",46,"315,000",3.78
Victor Barnes,"1,187",44,"356,000",3.71
Quincy Vincent,"1,805",65,"465,500",3.60
Ben Hall,"1,303",46,"345,500",3.53



### 📊 Conclusions Based on Manager Performance Table

1. **Oliver Taylor has the highest conversion rate (31.25%)**  
   - He closed 50 successful deals out of 160 attempts, making him the leader in efficiency.  
   - This could indicate that he works with higher-quality leads or has strong sales techniques.  

2. **Charlie Davis has the highest number of deals (2,797 deals, 148 successful)**  
   - However, his conversion rate (5.29%) is relatively low, which may suggest he works with lower-quality leads or has a weak strategy for closing deals.  

3. **Ulysses Adams generates the highest revenue from successful deals (€1,011,400)**  
   - He closed 141 deals out of 2,069 attempts (6.81% conversion rate).  
   - This means he works with higher-value offers but does not always close deals successfully.  

4. **Low conversion rates among several managers despite high deal volumes**  
   - For example, Kevin Parker handled 500 deals but closed only 40, resulting in an 8% conversion rate.  
   - Quincy Vincent handled 1,805 deals but closed only 65 (3.6% conversion rate).  

5. **Jane Smith and Cara Iverson have low success rates and conversion rates**  
   - Jane Smith handled 904 deals but only closed 31 (3.43%).  
   - Cara Iverson handled 1,033 deals but closed only 26 (2.52%) — the lowest conversion rate among the managers.  

### Recommendations  
- **Analyze Oliver Taylor’s high conversion rate** – his sales methods could potentially be adapted for other managers.  
- **Focus on improving the efficiency of Charlie Davis and Ulysses Adams** – they handle a large number of deals, but their conversion rates could be improved.  
- **Revise the strategy for managers with low conversion rates (Cara Iverson, Jane Smith)** – they may be receiving low-quality leads or require additional training.


In [56]:
oliver_deals = deals[deals['Deal Owner Name'] == 'Oliver Taylor']
oliver_quality_distribution = oliver_deals[
                      'Quality'].value_counts(normalize=True) * 100

oliver_contacts = contacts[contacts['Contact Owner Name'] == 'Oliver Taylor']
total_contacts = oliver_contacts.shape[0]

oliver_calls = calls[calls['Call Owner Name'] == 'Oliver Taylor']
total_calls = oliver_calls.shape[0]
call_status_distribution = oliver_calls[
                      'Call Status'].value_counts(normalize=True) * 100

oliver_campaigns = oliver_deals['Campaign'].value_counts(normalize=True) * 100

print(
    f"oliver_quality_distribution - {oliver_quality_distribution},\n"
    "====================\n"
    f"total_contacts - {total_contacts},\n"
    "====================\n"
    f"total_calls - {total_calls},\n"
    "====================\n"
    f"call_status_distribution - {call_status_distribution},\n"
    "====================\n"
    f"oliver_campaigns - {oliver_campaigns}"
)


oliver_quality_distribution - Quality
B - Medium           44.936709
C - Low              36.075949
A - High             12.658228
D - Non Target        5.696203
E - Non Qualified     0.632911
Name: proportion, dtype: float64,
total_contacts - 19,
total_calls - 10,
call_status_distribution - Call Status
Attended Dialled      90.0
Unattended Dialled    10.0
Name: proportion, dtype: float64,
oliver_campaigns - Campaign
Dis_DE                               13.675214
performancemax_digitalmarkt_ru_DE    11.965812
02.07.23wide_DE                       9.401709
youtube_shorts_DE                     9.401709
03.07.23women                         8.547009
24.09.23retargeting_DE                5.982906
12.07.2023wide_DE                     5.128205
07.07.23LAL_DE                        4.273504
12.09.23interests_Uxui_DE             4.273504
mu_DE                                 3.418803
15.07.23b_DE                          2.564103
05.07.23interests_DE                  1.709402
arina_DE       

### **Reasons for Oliver Taylor's High Conversion Rate**  

**Oliver’s Leads Are of Higher Quality**
- **81%** of Oliver's deals come from **B - Medium (44.9%)** and **C - Low (36.1%)** client categories.  
- **A - High (12.7%)** is also above the company average.  
- **Non-targeted leads are minimal** (**D - Non Target = 5.7%**, **E - Non Qualified = 0.6%**).  

🔍 **Insight:** Oliver receives a high-quality client base compared to other employees.  

**Oliver Manages Very Few Contacts (Only 19)**
- Other employees handle significantly more contacts but have lower conversion rates.  
- This may indicate **a more selective approach** to clients or **a lighter workload**.  

**Few Calls, but Highly Effective (Only 10 Calls)**
- **90% of calls → Attended Dialled (answered).**  
- Only **10% of calls go unanswered**.  
- The department's average success rate is much lower (**~50-60% successful calls**).  

🔍 **Insight:** Oliver makes fewer calls, but his clients almost always pick up, increasing his chances of closing deals.  

**Highly Effective Marketing Campaigns Drive Oliver’s Clients**

Top 3 lead sources:  
- **Dis_DE (13.7%)**  
- **performancemax_digitalmarkt_ru_DE (12%)**  
- **02.07.23wide_DE (9.4%)**  

These campaigns likely attract a **better-targeted audience**, improving conversion rates.  
Other employees rely on **less effective campaigns**.  

🔍 **Insight:** Oliver benefits from **high-conversion campaigns**, ensuring high-quality leads.  

🔍 **Summary conclusion:**

✅ Oliver receives the best clients (high share of B and A).

✅ He is assigned fewer but higher-quality contacts (19 contacts).

✅ Almost all of his calls are successful (90% response rate).

✅ He receives leads from the most effective advertising campaigns.


💡 Possible Explanation:
Oliver focuses on quality over quantity—he is assigned pre-qualified clients, spends less time on cold calls, but conducts them more effectively.

A deeper analysis could be conducted to examine how many contacts were assigned to other employees from the same advertising campaigns.

Let’s analyze the effectiveness of the sales department in terms of advertising campaigns, focusing on the number of processed deals, conversion rates, and total sales amounts.

In [57]:
campaign_deals = deals[deals['Campaign'] != 'Unknown'].groupby(
    'Campaign', observed=True)['Id'].nunique().reset_index(name='Total Deals')


successful_campaign_deals = deals[(deals['Stage'] == 'Payment Done') &
                            (deals['Campaign'] != 'Unknown')]\
                            .groupby('Campaign', observed=True)['Id']\
                            .nunique().reset_index(name='Successful Deals')

total_sales_campaign = deals[(deals['Stage'] == 'Payment Done') &
                             (deals['Campaign'] != 'Unknown')]\
                    .groupby('Campaign', observed=True)['Offer Total Amount']\
                    .sum().reset_index()

campaign_performance = pd.merge(campaign_deals,
                                successful_campaign_deals,
                                on='Campaign',
                                how='left')
campaign_performance = pd.merge(campaign_performance,
                                total_sales_campaign,
                                on='Campaign',
                                how='left')


campaign_performance['Successful Deals'] = campaign_performance[
    'Successful Deals'].fillna(0)
campaign_performance['Offer Total Amount'] = campaign_performance[
    'Offer Total Amount'].fillna(0)
campaign_performance['Conversion Rate'] = campaign_performance[
    'Successful Deals'] / campaign_performance['Total Deals'] * 100

campaign_performance_sorted = campaign_performance.sort_values(
    by='Successful Deals', ascending=False)

campaign_performance_sorted.columns = [
                                'Campaign', 'Total Deals', 'Successful Deals',
                                'Offer Total Amount', 'Conversion Rate']

formatted_campaign_performance_sorted = campaign_performance_sorted[
    ['Campaign', 'Total Deals', 'Successful Deals',
     'Offer Total Amount', 'Conversion Rate']].style.format({
    'Successful Deals': '{:,.0f}',
    'Offer Total Amount': '{:,.0f}',
    'Conversion Rate': '{:.2f}'
}).hide(axis='index')

display(formatted_campaign_performance_sorted)

Campaign,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
performancemax_digitalmarkt_ru_DE,2570,112,"840,900",4.36
youtube_shorts_DE,1596,53,"415,500",3.32
02.07.23wide_DE,940,52,"396,900",5.53
12.07.2023wide_DE,1530,48,"336,900",3.14
03.07.23women,592,31,"256,000",5.24
04.07.23recentlymoved_DE,728,31,"199,900",4.26
Dis_DE,566,30,"222,000",5.30
07.07.23LAL_DE,527,28,"243,500",5.31
12.09.23interests_Uxui_DE,514,27,"229,000",5.25
24.09.23retargeting_DE,471,17,"146,000",3.61


In [58]:
campaign_performance

,Campaign,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
0,01.04.23women_PL,31,0.0,0.0,0.000000
1,02.05.24test_DE,120,1.0,11000.0,0.833333
2,02.07.23wide_DE,940,52.0,396900.0,5.531915
3,02.08.23interests_DE,8,0.0,0.0,0.000000
4,03.07.23women,592,31.0,256000.0,5.236486
...,...,...,...,...,...
148,work_wr,6,0.0,0.0,0.000000
149,workingin_DE,48,0.0,0.0,0.000000
150,yo_DE,13,0.0,0.0,0.000000
151,youtube_shorts_DE,1596,53.0,415500.0,3.320802


In [59]:
top_10_campaigns = campaign_performance_sorted.head(10)
top_10_campaigns_cleaned = top_10_campaigns.copy()
top_10_campaigns_cleaned['Campaign'] = top_10_campaigns_cleaned['Campaign'].str[:10]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_10_campaigns_cleaned['Campaign'],
    y=top_10_campaigns_cleaned['Total Deals'],
    name='Total Deals',
    marker_color='skyblue'
))

fig.add_trace(go.Bar(
    x=top_10_campaigns_cleaned['Campaign'],
    y=top_10_campaigns_cleaned['Successful Deals'],
    name='Successful Deals',
    marker_color='lightgreen'
))

scaled_conversion_rate = [rate * 40 for rate in top_10_campaigns_cleaned['Conversion Rate']]

fig.add_trace(go.Scatter(
    x=top_10_campaigns_cleaned['Campaign'],
    y=scaled_conversion_rate,
    mode='lines+markers+text',
    name='Conversion Rate (%) (scaled)',
    text=[f'{rate:.1f}%' for rate in top_10_campaigns_cleaned['Conversion Rate']],
    textposition='top center',
    marker=dict(color='red', size=8),
    line=dict(color='red', width=2)
))

fig.update_layout(
    title=dict(
        text='Top 10 Campaigns: Total Deals, Successful Deals, and Conversion Rate',
        x=0.5, xanchor='center'
    ),
    xaxis_title='',
    yaxis_title='Total Deals and Successful Deals',
    yaxis2=dict(title='Conversion Rate (%)', overlaying='y', side='right'),
    legend=dict(x=0.5, y=1.07, xanchor='center', orientation='h'),
    barmode='group',
    xaxis_tickangle=0,
    height=800,
    width=1200
)

fig.show()

#### **Key Insights**:  
- **Top Campaign by Total Deals**: `performancemax_digitalmarkt_ru_DE` generated the highest number of deals, but its conversion rate (4.4%) is **not the highest**.  
- **Best Conversion Rate**: Campaign `02.07.23wide_DE` achieved the highest conversion rate (5.5%) despite fewer total deals, suggesting **efficient targeting**.  
- **Volume vs. Quality**: High-deal campaigns often underperform in conversion rates, highlighting a potential need to **optimize targeting strategies** or **improve lead quality**.  




---



To evaluate the performance of the sales department, it is important to understand which marketing campaigns and lead sources bring the highest conversion and profit. To do this, we group data by campaigns and sources, calculating the average and median time to close deals, the number and success of deals, as well as the conversion rate; we evaluate financial indicators such as the average check and total revenue.

In [60]:
deals_contacts_noTest = deals_contacts[deals_contacts['Source'] != 'Test']
campaign_source_analysis = deals_contacts_noTest.groupby(
    ['Campaign', 'Source']
).agg(
    Mean_Time_to_Close=('Duration', 'mean'),
    Median_Time_to_Close=('Duration', 'median'),
    Total_Deals=('Stage', 'count'),
    Successful_Deals=('Stage', lambda x: (x == 'Payment Done').sum()),
    Conversion_Rate=('Stage', lambda x: (
        (x == 'Payment Done').sum() / len(x) * 100 if len(x) > 0 else 0
    ))
).reset_index()

campaign_detailed_analysis = deals.groupby(
    'Campaign'
).agg(
    Total_Deals=('Stage', 'count'),
    Converted_Deals=('Stage', lambda x: (x == 'Payment Done').sum()),
    Conversion_Rate=('Stage', lambda x: (
        (x == 'Payment Done').sum() / len(x) * 100 if len(x) > 0 else 0
    )),
    Total_Sales=('Initial Amount Paid', 'sum'),
    Avg_Deal_Amount=('Initial Amount Paid', lambda x: x[x > 0].mean()),
    Avg_Time_to_Close=('Closing Date', lambda x: (
        (x - deals.loc[x.index, 'Created Time']).dt.days.mean()
    ))
).reset_index()

stage_distribution_campaign = deals.pivot_table(
    index='Campaign',
    columns='Stage',
    values='Id',
    aggfunc='count',
    fill_value=0
).reset_index()

In [61]:
campaign_source_analysis.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Campaign              172 non-null    object 
 1   Source                172 non-null    object 
 2   Mean_Time_to_Close    172 non-null    float64
 3   Median_Time_to_Close  172 non-null    float64
 4   Total_Deals           172 non-null    int64  
 5   Successful_Deals      172 non-null    int64  
 6   Conversion_Rate       172 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.5+ KB


In [62]:
campaign_source_analysis = deals_contacts_noTest.groupby(['Campaign', 'Source']).agg(
    Mean_Time_to_Close=('Duration', 'mean'),
    Median_Time_to_Close=('Duration', 'median'),
    Total_Deals=('Stage', 'count')).reset_index()
campaign_source_analysis = campaign_source_analysis.merge(
    campaign_performance[['Campaign', 'Successful Deals', 'Conversion Rate']],
    on='Campaign',
    how='left'
)
campaign_source_analysis

,Campaign,Source,Mean_Time_to_Close,Median_Time_to_Close,Total_Deals,Successful Deals,Conversion Rate
0,01.04.23women_PL,Facebook Ads,11.750000,5.5,24,0.0,0.000000
1,02.07.23wide_DE,Facebook Ads,18.783668,5.0,698,52.0,5.531915
2,02.08.23interests_DE,Facebook Ads,13.250000,4.5,8,0.0,0.000000
3,03.07.23women,Facebook Ads,20.513575,4.0,442,31.0,5.236486
4,04.07.23recentlymoved_DE,Facebook Ads,16.408348,4.0,551,31.0,4.258242
...,...,...,...,...,...,...,...
167,workingin_DE,Telegram posts,19.153846,2.0,39,0.0,0.000000
168,yo_DE,SMM,13.100000,4.5,10,0.0,0.000000
169,youtube_shorts_DE,Google Ads,61.000000,61.0,1,53.0,3.320802
170,youtube_shorts_DE,Youtube Ads,13.199833,2.0,1196,53.0,3.320802


In [90]:
campaign_detailed_analysis['Campaign'] = campaign_detailed_analysis['Campaign']\
                                                                    .str[:10]
campaign_source_analysis['Campaign'] = campaign_source_analysis['Campaign']\
                                                                    .str[:10]

fig1 = px.bar(
    campaign_detailed_analysis,
    x='Campaign',
    y='Avg_Deal_Amount',
    labels={'Avg_Deal_Amount': 'Average Deal Amount'},
    color='Avg_Deal_Amount',
    color_continuous_scale='Teal'
)
fig1.update_layout(
    title=dict(
        text="Average Deal Amount by Campaign",
        x=0.5,
        xanchor="center",
        font=dict(size=18)),
    xaxis_title="",
    xaxis=dict(tickangle=-90),
    width=1200,
    height=360
)

fig2 = px.scatter(
    campaign_source_analysis[campaign_source_analysis.Total_Deals > 50],
    x='Campaign',
    y='Mean_Time_to_Close',
    size='Total_Deals',
    color='Conversion Rate',
    hover_name='Source',
    labels={
        'Mean_Time_to_Close': 'Mean Time to Close (days)',
        'Total_Deals': 'Total Deals',
        'Conversion Rate': 'Conversion Rate (%)'
    },
    color_continuous_scale='Viridis',
    size_max=30
)

fig2.update_layout(
    title=dict(
        text="Mean Time to Close by Campaign and Source",
        x=0.5,
        xanchor="center",
        font=dict(size=18)),
    xaxis_title="",
    yaxis_title="Mean Time to Close (days)",
    xaxis=dict(tickangle=-90),
    width=1200,
    height=360
)

fig1.show()
fig2.show()

### 🔍 Conclusion
1. Some campaigns generate a significantly higher number of leads compared to others—these campaigns are worth analyzing in detail and scaling.
2. There are campaigns with a long deal closing cycle, which may require additional touches with clients.
3. High conversion is not always associated with a short closing time, which indicates different sales mechanics.
4. Optimizing campaigns with low conversion and long sales can improve marketing efficiency.
5. Identifying successful combinations of campaigns and sources will help focus on the most profitable strategies.



---



### Analysis of Payments and Products:
- Examine the distribution of payment types and their impact on the success of deals.
---

In [64]:
filtered_deals = deals[deals['Payment Type'].astype(str) != 'Unknown']

total_payments = (
    filtered_deals.groupby('Payment Type', observed=True)['Id']
    .nunique()
    .reset_index(name='Total Deals')
)

successful_payments = (
    filtered_deals[filtered_deals['Stage'] == 'Payment Done']
    .groupby('Payment Type', observed=True)['Id']
    .nunique()
    .reset_index(name='Successful Deals')
)

payment_performance = pd.merge(
    total_payments, successful_payments, on='Payment Type', how='left'
)

payment_performance['Successful Deals'] = (
    payment_performance['Successful Deals'].fillna(0)
)

payment_performance['Conversion Rate'] = (
    payment_performance['Successful Deals']
    / payment_performance['Total Deals'] * 100
).round(1)

fig = px.bar(
    payment_performance,
    x='Payment Type',
    y=['Total Deals', 'Successful Deals'],
    barmode='group',
    color_discrete_map={
        'Total Deals': 'skyblue',
        'Successful Deals': 'lightgreen'
    },
    title='Payment Performance: Total Deals, Successful Deals, '
          'and Conversion Rate',
    text_auto='.2s'
)

for trace in fig.data:
    if 'bar' in trace.type:
        trace.textposition = 'outside'

fig.add_trace(
    go.Scatter(
        x=payment_performance['Payment Type'],
        y=payment_performance['Conversion Rate'],
        mode='lines+markers+text',
        name='Conversion Rate',
        marker=dict(color='red', size=8),
        text=payment_performance['Conversion Rate'].astype(str) + '%',
        textposition='top left'
    )
)

fig.update_layout(
    title_x=0.5,
    yaxis=dict(title='Number of Deals'),
    yaxis2=dict(
        title='Conversion Rate (%)',
        overlaying='y',
        side='right',
        showgrid=False
    ),
    xaxis_title='',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        xanchor="center",
        y=1.1
    ),
    template='plotly_white',
)

fig.show()

### Insights from the Graph:
1. The highest number of deals falls under the *Recurring Payments* category (340 deals), followed by *One Payment* (140 deals), while *Reservation* has the fewest deals (5).
2. Successful deals are also most concentrated in the *Recurring Payments* category (250 successful deals) and *One Payment* (110), while *Reservation* has only 1 successful deal.
3. Conversion Rate:
   - The highest is for *One Payment* – 82.4%, indicating a high likelihood of successful deal completion.
   - For *Recurring Payments*, it is lower – 73%, which may suggest challenges with recurring payments.
   - The lowest is for *Reservation* – 20%, indicating a high likelihood of reservation cancellations.
4. Overall Trend: Despite the high number of deals in *Recurring Payments*, their conversion rate is lower than that of *One Payment*. This could be related to the subscription process or additional factors affecting payments.

📊 Recommendation: It is worth analyzing the reasons for the lower conversion rates in *Recurring Payments* and *Reservation* — there may be factors hindering successful deal completion.

---
### Analyze the popularity and success rates of different products and types of training.
---

**Let’s create a table showing the calculation of metrics by training type:**
 - Number of deals for each training type,
 - Number of successful deals for each training type,
 - Total payment amount for successful deals by training type (with status "Payment Done"),
 - Conversion rate for each training type)

In [65]:
filtered_deals = deals[deals['Education Type'].isin(['Morning', 'Evening'])]

education_deals = (
    filtered_deals.groupby('Education Type', observed=True)['Id']
    .nunique()
    .reset_index(name='Total Deals')
)

successful_education_deals = (
    filtered_deals[filtered_deals['Stage'] == 'Payment Done']
    .groupby('Education Type', observed=True)['Id']
    .nunique()
    .reset_index(name='Successful Deals')
)

total_sales_education = (
    filtered_deals[filtered_deals['Stage'] == 'Payment Done']
    .groupby('Education Type', observed=True)['Offer Total Amount']
    .sum()
    .reset_index()
)

education_performance = pd.merge(
    education_deals, successful_education_deals, on='Education Type', how='left'
)
education_performance = pd.merge(
    education_performance, total_sales_education, on='Education Type', how='left'
)

education_performance[['Successful Deals',
                       'Offer Total Amount']] = education_performance[
    ['Successful Deals', 'Offer Total Amount']
].fillna({'Successful Deals': 0, 'Offer Total Amount': 0})

education_performance['Conversion Rate'] = (
    education_performance['Successful Deals'] / \
    education_performance['Total Deals'] * 100
)

education_performance_sorted = education_performance.sort_values(
    by='Successful Deals', ascending=False
)

education_performance_sorted.columns = [
    'Education Type', 'Total Deals', 'Successful Deals',
    'Offer Total Amount', 'Conversion Rate'
]

formatted_education_performance_sorted = education_performance_sorted[
    ['Education Type', 'Total Deals', 'Successful Deals',
     'Offer Total Amount', 'Conversion Rate']
].style.format({
    'Successful Deals': '{:,.0f}',
    'Offer Total Amount': '{:,.0f}',
    'Conversion Rate': '{:.2f}'
}).hide(axis='index')

display(formatted_education_performance_sorted)

Education Type,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
Morning,2984,668,"5,713,800",22.39
Evening,406,170,"626,300",41.87


🔍 **Conclusion:**

Based on the data in the table, we can see that the Morning training type attracted the most deals (2984) and generated the highest sales amount (5,713,800), but its conversion rate was 22.39%. On the other hand, the Evening training type showed less overall activity with 406 deals and a sales amount of 626.30, but it still demonstrated a significantly higher conversion rate of 41.87%. This indicates that evening classes, although less popular, are more effective in converting leads into successful deals.

Let’s create a table showing the calculation of metrics by products, considering training types:

- Number of deals for each product
- Number of successful deals for each product (with status "Payment Done")
- Total payment amount for successful deals by product
- Conversion rate for each product

In [66]:
product_education_deals = (
    deals.groupby(['Product', 'Education Type'], observed=True)['Id']
    .nunique()
    .reset_index(name='Total Deals')
)

successful_product_education_deals = (
    deals[deals['Stage'] == 'Payment Done']
    .groupby(['Product', 'Education Type'], observed=True)['Id']
    .nunique()
    .reset_index(name='Successful Deals')
)

total_sales_product_education = (
    deals[deals['Stage'] == 'Payment Done']
    .groupby(['Product', 'Education Type'], observed=True)['Offer Total Amount']
    .sum()
    .reset_index()
)

product_education_performance = pd.merge(
    product_education_deals, successful_product_education_deals,
    on=['Product', 'Education Type'], how='left'
)
product_education_performance = pd.merge(
    product_education_performance, total_sales_product_education,
    on=['Product', 'Education Type'], how='left'
)

product_education_performance.fillna(0, inplace=True)

product_education_performance['Conversion Rate'] = (
    product_education_performance['Successful Deals'] /
    product_education_performance['Total Deals'] * 100)

product_education_performance_sorted = product_education_performance.sort_values(
    by='Successful Deals', ascending=False
)

formatted_product_education_performance_sorted = (
    product_education_performance_sorted[
        ['Product', 'Education Type', 'Total Deals',
         'Successful Deals', 'Offer Total Amount', 'Conversion Rate']
    ]
    .style.format({
        'Total Deals': '{:,.0f}',
        'Successful Deals': '{:,.0f}',
        'Offer Total Amount': '{:,.0f}',
        'Conversion Rate': '{:.2f}'
    })
    .hide(axis='index')
)

display(formatted_product_education_performance_sorted)

Product,Education Type,Total Deals,Successful Deals,Offer Total Amount,Conversion Rate
Digital Marketing,Morning,"1,618",361,"3,522,200",22.31
UX/UI Design,Morning,815,170,"1,608,400",20.86
Web Developer,Morning,542,137,"583,200",25.28
Digital Marketing,Evening,253,113,"412,300",44.66
UX/UI Design,Evening,152,57,"214,000",37.50
Web Developer,Evening,1,0,0,0.00


🔍 **Conclusion:**

- Utrenniye kursy privlekayut bol'she vsego sdelok dlya vsekh produktov, pri etom kursy po Digital Marketing naiboleye populyarny i uspeshny, oni imeyut 1618 sdelok, iz kotorykh 361 byli uspeshnymi, chto privelo k obshchey summe prodazh v 3,522,200 i koeffitsiyentu konversii 22.31%. - Vecherniye kursy po Digital Marketing takzhe pokazyvayut vysokiy koeffitsiyent konversii (44.66%) pri 253 sdelki i 113 uspeshnykh. - UX/UI Design i Web Developer kursy takzhe populyarny v utrenniye chasy, s 815 i 542 sdelkami sootvetstvenno, i uspeshnymi sdelkami v 170 i 137. - Vecherniye kursy po UX/UI Design imeyut vysokiy koeffitsiyent konversii (37.50%) s 152 sdelkami i 57 uspeshnymi.
Show more
640 / 5,000
- Morning courses attract the most deals across all products, with the Digital Marketing courses being the most popular and successful, with 1,618 deals, of which 361 were successful, resulting in a total of 3,522,200 sales and a conversion rate of 22.31%.
- Evening Digital Marketing courses also show a high conversion rate (44.66%) with 253 deals and 113 successful.
- UX/UI Design and Web Developer courses are also popular in the morning, with 815 and 542 deals respectively, and 170 and 137 successful deals.
- Evening UX/UI Design courses have a high conversion rate (37.50%) with 152 deals and 57 successful.

In [67]:
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        "Product Performance by Education Type",
        "Revenue Share by Product and Education Type"),
    specs=[[{"type": "xy"}, {"type": "domain"}]]
)

soft_colors_total = px.colors.qualitative.Pastel
soft_colors_successful = px.colors.qualitative.Pastel2
soft_colors_pie = px.colors.qualitative.Set3

fig_bar = px.bar(
    product_education_performance_sorted,
    x="Product",
    y="Total Deals",
    color="Education Type",
    text_auto=True,
    barmode="stack",
    color_discrete_sequence=soft_colors_total
)

threshold = 10
fig_bar2 = px.bar(
    product_education_performance_sorted,
    x="Product",
    y="Successful Deals",
    color="Education Type",
    barmode="stack",
    color_discrete_sequence=soft_colors_successful
)

for trace in fig_bar2.data:
    trace.text = [
        f"{y:.0f}" if y > threshold else ""
        for y in trace.y
    ]
    trace.textposition = "inside"

for trace in fig_bar.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig_bar2.data:
    fig.add_trace(trace, row=1, col=1)

pie_data = product_education_performance_sorted.groupby(
    ["Product", "Education Type"]
)["Offer Total Amount"].sum().reset_index()

fig_pie = px.pie(
    pie_data,
    values="Offer Total Amount",
    color = 'Product',
    names=[f"{prod} ({edu})" for prod, edu in zip(pie_data["Product"],
                                                  pie_data["Education Type"])],
    color_discrete_sequence = soft_colors_pie
)

for trace in fig_pie.data:
    fig.add_trace(trace, row=1, col=2)

fig.update_layout(
    height=500, width=1100,
    showlegend=True,
    template="plotly_white"
)

fig.show()

### Key Insights from the Graphs:

1. **Histogram**:  
   - Digital Marketing has the highest number of deals, especially in the morning format.  
   - UX/UI Design ranks second, with a clear preference for morning deals.  
   - Web Developer shows the lowest number of deals, particularly in the evening.  
   - Overall, the morning format is more popular across all products.

2. **Pie Chart**:  
   - The primary contribution to revenue comes from Digital Marketing (Morning) – 55.6%.  
   - UX/UI Design (Morning) also holds a significant share – 25.4%.  
   - Evening programs (especially Web Developer Evening) generate minimal revenue.  
   - The contribution of Web Developer (Morning) and Digital Marketing (Evening) is small compared to the leaders.  

### **Overall Conclusion**:  
- Morning programs are significantly more in demand and generate higher revenue.  
- Digital Marketing (Morning) is the leader in both the number of deals and financial contribution.  
- Evening programs require a strategy review due to their low popularity and profitability.



---



## Geographic Analysis:
1. Analyze the geographic distribution of deals by cities.
---

In [70]:
with open("city_data_google.json", "r") as json_file:
    city_data = json.load(json_file)

In [71]:
def get_city_info(city):
    info = city_data.get(city, {})
    return pd.Series({
        'longitude': info.get('longitude', None),
        'latitude': info.get('latitude', None),
        'country_ru': info.get('country', None),
        'city_ru': info.get('formatted_address', None)
    })

deals['City'] = deals['City'].fillna('Unknown')
deals[['longitude', 'latitude', 'country_ru', 'city_ru']] = deals['City'].apply(get_city_info)

In [72]:
deals.loc[deals.City != "Unknown"].head()

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,Content,Term,...,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,SLA Seconds,longitude,latitude,country_ru,city_ru
32,5805028000056714532,Ulysses Adams,NaT,NaN,Registered on Webinar,NaN,/webinar,webinar1906,NaN,invitation,...,NaN,NaN,5805028000044019127,Berlin,Unknown,NaN,13.404954,52.520007,Германия,Берлин
39,5805028000056731279,Ulysses Adams,NaT,NaN,Registered on Webinar,NaN,/webinar,NaN,NaN,NaN,...,NaN,NaN,5805028000054755989,Lahnstein,Unknown,NaN,7.609363,50.307827,Германия,Ланштайн
49,5805028000056683030,Charlie Davis,NaT,C - Low,Waiting For Payment,NaN,/eng/test,performancemax_eng_DE,_{region_name}_,NaN,...,1000.0,9000.0,5805028000056690015,Crailsheim,Unknown,1239.0,10.063357,49.133735,Германия,Крайльсхайм
50,5805028000056568397,Paula Underwood,NaT,NaN,Registered on Webinar,NaN,/webinar,NaN,NaN,NaN,...,NaN,NaN,5805028000020664131,Prenzlau,B1,NaN,13.862376,53.316844,Германия,Пренцлау
53,5805028000056558351,Ulysses Adams,NaT,C - Low,Waiting For Payment,NaN,/eng,NaN,NaN,NaN,...,1000.0,9000.0,5805028000056578244,Dortmund,B1,589.0,7.465298,51.513587,Германия,Дортмунд


In [73]:
deals['City'] = deals.City.replace('-', "Unknown")

In [74]:
deals.to_csv("deals_geo.csv", index=False)

In [75]:
deals_dashbord = deals[deals['Stage'] == 'Payment Done'].groupby('City').agg({'Stage': 'count'}) / deals.groupby('City').agg({'Stage': 'count'}) * 100
deals_dashbord[deals_dashbord.index == 'Frankfurt']


,Stage
City,
Frankfurt,20.588235


### Calculation of Statistics

In [76]:
geo_deals = deals[['Id', 'City']]
geo_deals_clean = geo_deals.dropna(subset=['City'])
deals_by_city = geo_deals_clean.groupby('City').size().reset_index(name='Number of Deals')

deals_by_city_sorted = deals_by_city.sort_values(by='Number of Deals', ascending=False)
deals_by_city_sorted

,City,Number of Deals
774,Unknown,17096
88,Berlin,264
521,München,88
288,Hamburg,78
570,Nürnberg,53
...,...,...
442,Liederbach am Taunus,1
443,Lierschied,1
444,Limbach-Oberfrohna,1
448,Linz,1


In [77]:
geo_data_deals = deals[["City", "longitude", "latitude", "country_ru",
                           "city_ru"]].drop_duplicates(subset=["City"]).dropna()

In [78]:
deals_by_city_sorted = deals_by_city_sorted.merge(geo_data_deals, on='City', how="inner")
deals_by_city_sorted

,City,Number of Deals,longitude,latitude,country_ru,city_ru
0,Berlin,264,13.404954,52.520007,Германия,Берлин
1,München,88,11.581981,48.135125,Германия,Мюнхен
2,Hamburg,78,9.987170,53.548828,Германия,Гамбург
3,Nürnberg,53,11.074564,49.454288,Германия,Нюрнберг
4,Leipzig,52,12.373075,51.339695,Германия,Лейпциг
...,...,...,...,...,...,...
864,Liederbach am Taunus,1,8.484905,50.127530,Германия,Лидербах
865,Lierschied,1,7.746058,50.170155,Германия,Лиршид
866,Limbach-Oberfrohna,1,12.753720,50.857566,Германия,Лимбах-Оберфрона
867,Linz,1,14.285918,48.306910,Австрия,Линц


In [79]:
deals_by_city_sorted = deals_by_city_sorted.merge(geo_data_deals, on='City', how="inner")
deals_by_city_sorted

,City,Number of Deals,longitude_x,latitude_x,country_ru_x,city_ru_x,longitude_y,latitude_y,country_ru_y,city_ru_y
0,Berlin,264,13.404954,52.520007,Германия,Берлин,13.404954,52.520007,Германия,Берлин
1,München,88,11.581981,48.135125,Германия,Мюнхен,11.581981,48.135125,Германия,Мюнхен
2,Hamburg,78,9.987170,53.548828,Германия,Гамбург,9.987170,53.548828,Германия,Гамбург
3,Nürnberg,53,11.074564,49.454288,Германия,Нюрнберг,11.074564,49.454288,Германия,Нюрнберг
4,Leipzig,52,12.373075,51.339695,Германия,Лейпциг,12.373075,51.339695,Германия,Лейпциг
...,...,...,...,...,...,...,...,...,...,...
864,Liederbach am Taunus,1,8.484905,50.127530,Германия,Лидербах,8.484905,50.127530,Германия,Лидербах
865,Lierschied,1,7.746058,50.170155,Германия,Лиршид,7.746058,50.170155,Германия,Лиршид
866,Limbach-Oberfrohna,1,12.753720,50.857566,Германия,Лимбах-Оберфрона,12.753720,50.857566,Германия,Лимбах-Оберфрона
867,Linz,1,14.285918,48.306910,Австрия,Линц,14.285918,48.306910,Австрия,Линц


In [80]:
bar_color = px.colors.qualitative.Pastel[0]

fig = go.Figure(data=[
    go.Bar(
        name='Number of Deals',
        x=deals_by_city_sorted['City'].head(10),
        y=deals_by_city_sorted['Number of Deals'].head(10),
        marker_color=bar_color  # Один цвет для всех баров
    )
])

fig.update_layout(
    title=dict(text="Top 10 Cities by Number of Deals", x=0.5),
    yaxis_title="Number of Deals",
    xaxis_tickangle=0
)

fig.show()

In [81]:
deals_by_city_sorted.to_csv("deals_by_city_sorted.csv", index=False)

In [82]:
deals_by_city_sorted

,City,Number of Deals,longitude_x,latitude_x,country_ru_x,city_ru_x,longitude_y,latitude_y,country_ru_y,city_ru_y
0,Berlin,264,13.404954,52.520007,Германия,Берлин,13.404954,52.520007,Германия,Берлин
1,München,88,11.581981,48.135125,Германия,Мюнхен,11.581981,48.135125,Германия,Мюнхен
2,Hamburg,78,9.987170,53.548828,Германия,Гамбург,9.987170,53.548828,Германия,Гамбург
3,Nürnberg,53,11.074564,49.454288,Германия,Нюрнберг,11.074564,49.454288,Германия,Нюрнберг
4,Leipzig,52,12.373075,51.339695,Германия,Лейпциг,12.373075,51.339695,Германия,Лейпциг
...,...,...,...,...,...,...,...,...,...,...
864,Liederbach am Taunus,1,8.484905,50.127530,Германия,Лидербах,8.484905,50.127530,Германия,Лидербах
865,Lierschied,1,7.746058,50.170155,Германия,Лиршид,7.746058,50.170155,Германия,Лиршид
866,Limbach-Oberfrohna,1,12.753720,50.857566,Германия,Лимбах-Оберфрона,12.753720,50.857566,Германия,Лимбах-Оберфрона
867,Linz,1,14.285918,48.306910,Австрия,Линц,14.285918,48.306910,Австрия,Линц


In [83]:
deals_geo = deals[deals['City'] != 'Unnamed'].dropna(subset="city_ru")

geometry = [Point(xy) for xy in zip(deals_geo["longitude"], deals_geo["latitude"])]

gdf = gpd.GeoDataFrame(deals_geo, geometry=geometry)

gdf.set_crs(epsg=4326, inplace=True)

display(gdf.head())

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,Content,Term,...,Offer Total Amount,Contact Name,City,Level of Deutsch,SLA Seconds,longitude,latitude,country_ru,city_ru,geometry
32,5805028000056714532,Ulysses Adams,NaT,NaN,Registered on Webinar,NaN,/webinar,webinar1906,NaN,invitation,...,NaN,5805028000044019127,Berlin,Unknown,NaN,13.404954,52.520007,Германия,Берлин,POINT (13.40495 52.52001)
39,5805028000056731279,Ulysses Adams,NaT,NaN,Registered on Webinar,NaN,/webinar,NaN,NaN,NaN,...,NaN,5805028000054755989,Lahnstein,Unknown,NaN,7.609363,50.307827,Германия,Ланштайн,POINT (7.60936 50.30783)
49,5805028000056683030,Charlie Davis,NaT,C - Low,Waiting For Payment,NaN,/eng/test,performancemax_eng_DE,_{region_name}_,NaN,...,9000.0,5805028000056690015,Crailsheim,Unknown,1239.0,10.063357,49.133735,Германия,Крайльсхайм,POINT (10.06336 49.13374)
50,5805028000056568397,Paula Underwood,NaT,NaN,Registered on Webinar,NaN,/webinar,NaN,NaN,NaN,...,NaN,5805028000020664131,Prenzlau,B1,NaN,13.862376,53.316844,Германия,Пренцлау,POINT (13.86238 53.31684)
53,5805028000056558351,Ulysses Adams,NaT,C - Low,Waiting For Payment,NaN,/eng,NaN,NaN,NaN,...,9000.0,5805028000056578244,Dortmund,B1,589.0,7.465298,51.513587,Германия,Дортмунд,POINT (7.4653 51.51359)


In [84]:
gdf.geometry.is_empty.sum()

0

In [85]:
center = gdf.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=4)

marker_cluster = MarkerCluster().add_to(m)

for _, row in gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=f"Index: {_}"
    ).add_to(marker_cluster)

m.save("deals_map_cluster.html")
m

Output hidden; open in https://colab.research.google.com to view.

### **Geographical Data Analysis: Key Findings & Recommendations**  

#### **Deal Distribution by Region**  
- The majority of deals are concentrated in **Central Europe**, especially **Germany**, as seen in clustering on the map.  
- **Berlin is the clear leader**, followed by **Munich and Hamburg** with significantly fewer deals.  
- Deals are also present in **Eastern Europe, the Middle East, and North America**, though in smaller volumes.  
- The **lowest deal activity** is observed in **North America and parts of Asia**, appearing as isolated points on the map.  

#### **Top Cities by Deal Volume**  
1️⃣ **Berlin** – Dominates the ranking with a large lead.  
2️⃣ **Munich & Hamburg** – Follow, but with significantly fewer deals.  
3️⃣ Other major cities (**Nuremberg, Leipzig, Düsseldorf, Frankfurt, Dresden, Dortmund, Cologne**) show much lower deal volumes, reinforcing business concentration in a few economic hubs.  


#### 🔍 **Overall Insights**  
- **Germany** is the dominant market, particularly **Berlin, Munich, and Hamburg**.  
- **Eastern Europe and the Middle East** show moderate activity but lag behind in total deals.  
- **The US and other global markets** have minimal presence, possibly due to lower demand or limited business expansion.  

####  **Recommendations for Market Expansion**  
- **Expand into neighboring European countries and the Middle East** to tap into emerging opportunities.  
- **Analyze demand in the US** to identify barriers to market entry.  
- **Strengthen presence in other German cities**, beyond Berlin, to balance market concentration.  

### 🔍 **Conclusion**  
The analysis highlights a **strong geographic focus on Germany**, with **growth potential in Eastern Europe and the Middle East**. Expanding strategically can help diversify deal distribution.

---
2. Investigate the impact of the level of German language proficiency on the success of deals in different cities.
---

Grouping successful deals by city and german proficiency


In [86]:
filtered_deals = deals.dropna(subset=['City', 'Level of Deutsch'])
deals_by_city_lang = (
    filtered_deals.groupby(['City', 'Level of Deutsch'])
    .agg(Total_Deals=('Id', 'count'),
         Successful_Deals=('Stage', lambda x: (x == 'Payment Done').sum()))
    .reset_index()
)

deals_by_city_lang['Success_Rate'] = (
    deals_by_city_lang['Successful_Deals'] / deals_by_city_lang['Total_Deals']
) * 100

print(deals_by_city_lang.head())
print(deals_by_city_lang["Success_Rate"].describe())

        City Level of Deutsch  Total_Deals  Successful_Deals  Success_Rate
0     Aachen               A2            1                 0           0.0
1     Aachen               B1            5                 2          40.0
2     Aachen          Unknown            4                 1          25.0
3      Aalen          Unknown            3                 0           0.0
4  Abensberg          Unknown            1                 0           0.0
count    1188.000000
mean       28.279459
std        39.878991
min         0.000000
25%         0.000000
50%         0.000000
75%        50.000000
max       100.000000
Name: Success_Rate, dtype: float64


In [87]:
deals_by_city_lang_filtered = deals_by_city_lang[
    (deals_by_city_lang['Level of Deutsch'] != "Unknown") &
    (deals_by_city_lang["City"] != "Unknown")
]

top_cities = deals_by_city_lang_filtered.groupby("City")["Total_Deals"]\
.sum().nlargest(10).index
deals_top_cities = deals_by_city_lang_filtered[
    deals_by_city_lang_filtered["City"].isin(top_cities)]

fig = px.bar(
    deals_top_cities,
    x="City",
    y="Success_Rate",
    color="Level of Deutsch",
    title="Success Rate by City and Level of German Language (Top 10 Cities)",
    labels={"Success_Rate": "Success Rate (%)", "City": "City",
            "Level of Deutsch": ""},
    barmode="group",
    text_auto=".2f"
)

fig.update_layout(
    title_x=0.5,
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="center",
    x=0.5,
    title_text=""
    )
)

fig.show()

In [88]:
deals_top_cities.to_csv("deals_top_cities.csv", index=False)

### 🔍 **Conclusion**
- High success rates among C1 and B2 speakers – in some cities (e.g. Duisburg and Köln) it is clear that level C1 shows 100% success, while B2 shows consistently high rates.
- Spread of success rates by level – success rates vary across cities. For example, in Munich, Leipzig and Köln, levels A2, B1 and B2 show high rates.
- Cities with the highest success rates – Duisburg and Hamburg stand out for their high success rates at certain language levels.
- Cities with an even distribution – in Berlin and Leipzig, success is distributed relatively evenly across all levels.
- Possible city dependence – cities with a high level of German (B2 and C1) show the best results, which may indicate the importance of language training in successful transactions.



---



# Machine learning
---

We implement a machine learning pipeline to predict call statuses based on historical data. The process includes data preprocessing, feature scaling, encoding categorical variables, splitting the dataset, training a Random Forest classifier, and evaluating the model's performance. By following these steps, we aim to develop an accurate and interpretable predictive model that can assist in analyzing call outcomes.

In [91]:
from typing import Dict, Union, Tuple,List
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [92]:
df = calls.replace("Unknown", pd.NA)

In [93]:
df['CONTACTID'].value_counts()

,count
CONTACTID,
5805028000003329100,94
5805028000007866898,70
5805028000008246089,70
5805028000001880249,66
5805028000002799040,57
...,...
5805028000010421242,1
5805028000035995829,1
5805028000010409166,1


In [94]:
df = df.drop(['CONTACTID'], axis=1)

In [95]:
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 21:
        return 'evening'
    else:
        return 'night'

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

time_col = df['Call Start Time']

df['month'] = time_col.dt.month
df['year'] = time_col.dt.year
df['hour'] = time_col.dt.hour

df['time_of_day'] = df['hour'].apply(get_time_of_day)
df['season'] = df['month'].apply(get_season)

df = df.set_index('Id')

In [96]:
def identify_categorical(df):
    cat_cols = [
        col for col in df.columns
        if pd.api.types.is_categorical_dtype(df[col])
           or pd.api.types.is_object_dtype(df[col])
    ]
    return cat_cols


def analyze_categorical(df: pd.DataFrame, cat_cols) -> None:
    for col in cat_cols:
        total_rows = len(df)
        missing_count = df[col].isna().sum()
        missing_percent = (missing_count / total_rows) * 100

        unique_vals = df[col].dropna().unique()
        unique_count = len(unique_vals)
        unique_percent = (unique_count / total_rows) * 100

        print(f"\n=== Analyzing Categorical Column: '{col}' ===")
        print(f"Total Rows: {total_rows}")
        print(f"Missing Values: {missing_count} ({missing_percent:.2f}%)")
        print(f"Unique Categories (excl. NaN): {unique_count} ({unique_percent:.2f}%)")

        data_col = df[col].astype('string')
        data_col = data_col.fillna("NaN")
        counts = data_col.value_counts(dropna=False)
        perc = (counts / counts.sum()) * 100
        summary_df = pd.DataFrame({"Count": counts, "Percent": perc}).T
        print(f"\nValue Distribution for column '{col}':\n")
        display(summary_df)

analyze_categorical(df, identify_categorical(df))


=== Analyzing Categorical Column: 'Call Owner Name' ===
Total Rows: 92617
Missing Values: 0 (0.00%)
Unique Categories (excl. NaN): 33 (0.04%)

Value Distribution for column 'Call Owner Name':



<ipython-input-96-b77d15aea1d7>:4: DeprecationWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead



Call Owner Name,Yara Edwards,Julia Nelson,Ian Miller,Charlie Davis,Diana Evans,Ulysses Adams,Amy Green,Victor Barnes,Kevin Parker,Nina Scott,...,Fiona Jackson,Rachel White,Xander Dean,Ethan Harris,Hannah Lee,Wendy Clark,Bob Brown,Oliver Taylor,Tina Zhang,Laura Quinn
Count,8532,7213,7027,6943,6713,5961,5575,5361,5359,5317,...,451,425,294,277,175,162,98,10,5,2
Percent,9.212132,7.787987,7.58716,7.496464,7.248129,6.436183,6.019413,5.788354,5.786195,5.740847,...,0.486952,0.458879,0.317436,0.299081,0.18895,0.174914,0.105812,0.010797,0.005399,0.002159



=== Analyzing Categorical Column: 'Call Type' ===
Total Rows: 92617
Missing Values: 0 (0.00%)
Unique Categories (excl. NaN): 3 (0.00%)

Value Distribution for column 'Call Type':



Call Type,Outbound,Missed,Inbound
Count,83804,5742,3071
Percent,90.484468,6.199726,3.315806


...

Scheduled in CRM,0,NaN,1
Count,83668,8813,136
Percent,90.337627,9.515532,0.146841



=== Analyzing Categorical Column: 'time_of_day' ===
Total Rows: 92617
Missing Values: 0 (0.00%)
Unique Categories (excl. NaN): 4 (0.00%)

Value Distribution for column 'time_of_day':



time_of_day,afternoon,morning,evening,night
Count,59699,18485,14235,198
Percent,64.457929,19.958539,15.369749,0.213784



=== Analyzing Categorical Column: 'season' ===
Total Rows: 92617
Missing Values: 0 (0.00%)
Unique Categories (excl. NaN): 4 (0.00%)

Value Distribution for column 'season':



season,spring,winter,autumn,summer
Count,33692,25966,18890,14069
Percent,36.377771,28.03589,20.395824,15.190516


In [97]:
status_mapping = {
    'Scheduled Attended Delay': 'Attended Dialled',
    'Scheduled Attended': 'Attended Dialled',
    'Scheduled Unattended Delay': 'Unattended Dialled',
    'Scheduled Unattended': 'Unattended Dialled',
    'Scheduled': 'Unattended Dialled',
    'Cancelled': 'Missed'
}

df['Call Status'] = df['Call Status'].replace(status_mapping)

In [99]:
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

cleaned_df = df[numerical_columns + ['Call Owner Name', 'Call Type',
                                     'Call Status', 'season', 'time_of_day']]
cleaned_df = cleaned_df.dropna()
cleaned_df = cleaned_df.drop(['Call Type'], axis=1)
cleaned_df

,Call Duration (in seconds),month,year,hour,Call Owner Name,Call Status,season,time_of_day
Id,,,,,,,,
5805028000000805001,171.0,6,2023,8,John Doe,Received,summer,morning
5805028000000768006,28.0,6,2023,8,John Doe,Attended Dialled,summer,morning
5805028000000764027,24.0,6,2023,8,John Doe,Attended Dialled,summer,morning
5805028000000787003,6.0,6,2023,9,John Doe,Attended Dialled,summer,morning
5805028000000768019,11.0,6,2023,9,John Doe,Attended Dialled,summer,morning
...,...,...,...,...,...,...,...,...
5805028000056889515,6.0,6,2024,15,Ulysses Adams,Attended Dialled,summer,afternoon
5805028000056875317,8.0,6,2024,15,Victor Barnes,Attended Dialled,summer,afternoon
5805028000056832495,5.0,6,2024,15,Kevin Parker,Attended Dialled,summer,afternoon


In [100]:
numeric_columns = [
    col for col in cleaned_df.select_dtypes(include=[np.number])\
    .columns.tolist() if col != 'Call Status']
categorical_columns = [
    col for col in cleaned_df.select_dtypes(exclude=[np.number])\
    .columns.tolist() if col != 'Call Status']

In [101]:
scaler = MinMaxScaler()
cleaned_df[numeric_columns] = scaler.fit_transform(cleaned_df[numeric_columns])

final_df = pd.get_dummies(cleaned_df, columns=categorical_columns, drop_first=True, dtype=int)

final_df.head()

,Call Duration (in seconds),month,year,hour,Call Status,Call Owner Name_Amy Green,Call Owner Name_Ben Hall,Call Owner Name_Bob Brown,Call Owner Name_Cara Iverson,Call Owner Name_Charlie Davis,...,Call Owner Name_Wendy Clark,Call Owner Name_Xander Dean,Call Owner Name_Yara Edwards,Call Owner Name_Zachary Foster,season_spring,season_summer,season_winter,time_of_day_evening,time_of_day_morning,time_of_day_night
Id,,,,,,,,,,,,,,,,,,,,,
5805028000000805001,0.022426,0.454545,0.0,0.347826,Received,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5805028000000768006,0.003672,0.454545,0.0,0.347826,Attended Dialled,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5805028000000764027,0.003148,0.454545,0.0,0.347826,Attended Dialled,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5805028000000787003,0.000787,0.454545,0.0,0.391304,Attended Dialled,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5805028000000768019,0.001443,0.454545,0.0,0.391304,Attended Dialled,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [102]:
target_column = 'Call Status'
X = final_df.drop(columns=[target_column])
y = final_df[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
class_report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.9079
Confusion Matrix:
[[13612     0   320     0]
 [    0   577     0   649]
 [  536     0    68     0]
 [    0   199     0  2547]]
Classification Report:
                    precision    recall  f1-score   support

  Attended Dialled       0.96      0.98      0.97     13932
            Missed       0.74      0.47      0.58      1226
          Received       0.18      0.11      0.14       604
Unattended Dialled       0.80      0.93      0.86      2746

          accuracy                           0.91     18508
         macro avg       0.67      0.62      0.64     18508
      weighted avg       0.90      0.91      0.90     18508



### **Conclusions**

The trained Random Forest model demonstrates a solid predictive capability, achieving a satisfactory accuracy score. The confusion matrix and classification report provide insights into the model’s strengths and weaknesses, highlighting areas where predictions are more reliable and where misclassifications occur. These results suggest that the model effectively distinguishes between different call statuses, but further improvements could be made by fine-tuning hyperparameters, balancing the dataset, or incorporating additional features.